<center><h1 style="color:#D4AF37"> ⚡⚡ AEMR Data Analysis ⚡⚡</h1>

<img src = "https://images.squarespace-cdn.com/content/v1/551972d8e4b0d571edc2e2c8/8f3abd33-0916-4415-9b24-593eea1d1e3c/Screen+Shot+2022-04-20+at+12.41.24+PM.png">

  


<h1 style="color:#D4AF37"> What's the Business Problem? 💰</h1>

The American Energy Market Regulator (AEMR) is responsible for looking after the
United States of America’s domestic energy network. The regulator’s responsibility is to
ensure that America’s energy network remains reliable with minimal disruptions, which
are known as outages. 

There are four key types of outages:

● Consequential

● Forced 

● Opportunistic 

● Planned 

Recently, the AEMR management team has been increasingly aware of a large number
of energy providers that submitted outages over the 2016 and 2017 calendar years. The
management team has expressed a desire to have the following two areas of concern
addressed:

<b> A) Energy Stability and Market Outages
    <p>
B) Energy Losses and Market Reliability </b>


In [2]:
%%capture
!pip install ipython-sql sqlalchemy
import sqlalchemy
sqlalchemy.create_engine("sqlite:///AEMR.db")
%load_ext sql
%sql sqlite:///AEMR.db

In [1]:
%%js
require(['notebook/js/codecell'], function (codecell) {
    codecell.CodeCell.options_default.highlight_modes['magic_text/x-mssql'] = { 'reg': [/%?%sql/] };
    Jupyter.notebook.events.one('kernel_ready.Kernel', function () {
        Jupyter.notebook.get_cells().map(function (cell) {
            if (cell.cell_type == 'code') { cell.auto_highlight(); }
        });
    });
});

<IPython.core.display.Javascript object>

<h3 style="color:#D4AF37"> ⚡ Part I. Energy Stability & Market Outages ⚡ </h3>

Energy stability is one of the key themes the AEMR management team cares about. To ensure energy security and reliability, AEMR needs to understand the following:
<p>
<b>

<li> What are the most common outage types and how long do they tend to last? 
<li> How frequently do the outages occur? 
<li> Are there any energy providers which have more outages than their peers which may be indicative of being unreliable? 
    
<p>



In [17]:
%%sql
SELECT COUNT(*) as Total_Number_Outages,
Outage_Reason,
Year
FROM AEMR_Outage_Table
WHERE Status='Approved'
GROUP BY Outage_Reason, Year

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Outage_Reason,Year
181,Consequential,2016
127,Consequential,2017
1264,Forced,2016
1622,Forced,2017
106,Opportunistic Maintenance (Planned),2016
102,Opportunistic Maintenance (Planned),2017
380,Scheduled (Planned),2016
320,Scheduled (Planned),2017


In [5]:
%%sql
SELECT Year, Month, COUNT(*) as Total_Number_Outages
FROM AEMR_Outage_Table
WHERE Status='Approved'
GROUP BY Year, Month

 * sqlite:///AEMR.db
Done.


Year,Month,Total_Number_Outages
2016,1,191
2016,2,227
2016,3,136
2016,4,134
2016,5,174
2016,6,168
2016,7,147
2016,8,194
2016,9,124
2016,10,124


We've now identified a specific outage type is rapidly increasing from 2016 to 2017. However, frequency is just one aspect we care about. We also care about the <b> `duration` </b> of our ouatges.

In other words, if an outage is very short, we aren't as concerned. However, if the outage is very long, this then has the risk of threatening our energy supplies. We want to identify the problematic energy providers here. 

In [49]:
%%sql
SELECT Participant_Code, Outage_Reason, Year, Count(*) as Total_Number_Outage_Events, ROUND(AVG((ABS(JULIANDAY(End_Time)-JULIANDAY(Start_Time)))),2) as Average_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status='Approved' AND Outage_Reason='Forced'
GROUP BY Year, Participant_Code, Year
ORDER BY Total_Number_Outage_Events DESC, Outage_Reason, Year
LIMIT 10

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Duration_In_Days
AURICON,Forced,2017,490,0.07
GW,Forced,2016,317,0.38
GW,Forced,2017,227,1.06
AURICON,Forced,2016,208,0.07
AUXC,Forced,2016,206,0.08
MELK,Forced,2017,177,2.28
TRMOS,Forced,2017,172,0.42
MELK,Forced,2016,157,0.83
PUG,Forced,2017,135,0.25
AUXC,Forced,2017,120,0.02



We've identified participants who are having many outages, as well as participants who have been offline for the longest durations.

It's important to classify our participants accordingly based on reliability metrics of uptime.

Let's classify a participant based off the following criteria:
<b>
<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day)</li>
<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours </li>
<li> Low Risk - On average, the participant is unavailable for less than 12 Hours</li> 
</b>

In [65]:
%%sql
WITH MY_CTE_1 AS (
SELECT Participant_Code, Outage_Reason, Year, Count(*) as Total_Number_Outage_Events, ROUND(AVG((ABS(JULIANDAY(End_Time)-JULIANDAY(Start_Time)))),2) as Average_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status='Approved' AND Outage_Reason='Forced'
GROUP BY Year, Participant_Code, Year
ORDER BY Total_Number_Outage_Events DESC, Outage_Reason, Year
)
SELECT * ,
CASE
     WHEN Average_Duration_In_Days > 1 THEN 'High Risk'
     WHEN Average_Duration_In_Days BETWEEN 0.5 AND 1 THEN 'Medium Risk'
     WHEN Average_Duration_In_Days BETWEEN 0 AND 0.5 THEN 'Low Risk'
     ELSE 'N/A'
END AS Risk_Classification
FROM MY_CTE_1




 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Duration_In_Days,Risk_Classification
AURICON,Forced,2017,490,0.07,Low Risk
GW,Forced,2016,317,0.38,Low Risk
GW,Forced,2017,227,1.06,High Risk
AURICON,Forced,2016,208,0.07,Low Risk
AUXC,Forced,2016,206,0.08,Low Risk
MELK,Forced,2017,177,2.28,High Risk
TRMOS,Forced,2017,172,0.42,Low Risk
MELK,Forced,2016,157,0.83,Medium Risk
PUG,Forced,2017,135,0.25,Low Risk
AUXC,Forced,2017,120,0.02,Low Risk




Let's add two additional criteria to our classification considering `Total Number of Outage Events` and `Outage Type`.

summarised below:

<b>
<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day) OR the Total Number of Outage Events > 20 </li>
<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours OR the Total Number of Outage Events is Between 10 and 20 </li>
<li> Low Risk - On average, the participant is unavailable for less than 12 Hours OR the Total Number of Outage Events < 10 </li> 
<li> If Outage Type is not forced, then N/A
</b>


In [17]:
%%sql
WITH MY_CTE_2 AS (
SELECT Participant_Code, Outage_Reason, Year, Count(*) as Total_Number_Outage_Events, ROUND(AVG(ABS(JULIANDAY(End_Time)-JULIANDAY(Start_Time))),2) as Average_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status='Approved'
GROUP BY Year, Outage_Reason, Participant_Code
ORDER BY Total_Number_Outage_Events DESC
)
SELECT * ,
CASE
     WHEN Average_Duration_In_Days > 1 AND Outage_Reason = 'Forced' OR Total_Number_Outage_Events > 20 AND Outage_Reason = 'Forced' THEN 'High Risk'
     WHEN Average_Duration_In_Days BETWEEN 0.5 AND 1 AND Outage_Reason = 'Forced' OR Total_Number_Outage_Events BETWEEN 10 AND 20 AND Outage_Reason = 'Forced' THEN 'Medium Risk'
     WHEN Average_Duration_In_Days BETWEEN 0 AND 0.5 AND Outage_Reason = 'Forced' OR Total_Number_Outage_Events BETWEEN 0 AND 10 AND Outage_Reason = 'Forced' THEN 'Low Risk'
     ELSE 'N/A'
END AS Risk_Classification
FROM MY_CTE_2

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Duration_In_Days,Risk_Classification
AURICON,Forced,2017,490,0.07,High Risk
GW,Forced,2016,317,0.38,High Risk
GW,Forced,2017,227,1.06,High Risk
AURICON,Forced,2016,208,0.07,High Risk
AUXC,Forced,2016,206,0.08,High Risk
MELK,Forced,2017,177,2.28,High Risk
TRMOS,Forced,2017,172,0.42,High Risk
MELK,Forced,2016,157,0.83,High Risk
PUG,Forced,2017,135,0.25,High Risk
AUXC,Forced,2017,120,0.02,High Risk


<h3 style="color:#D4AF37"> ⚡ Part II. Energy Losses & Market Reliability ⚡ </h3>

In a situation where the outages are Forced, the energy provider intended to provide energy but are unable to provide energy and are forced offline. <b style="color:Maroon">If many energy providers are forced offline at the same time it could cause an energy security risk that AEMR needs to mitigate. </b>

To ensure this doesn’t happen, the AEMR is interested in exploring the following questions:

<li> Of the outage types in 2016 and 2017, what percent were Forced Outage(s)?
<li> What was the average duration for a forced outage during both 2016 and 2017? Have we seen an increase in the average duration of forced outages?
<li> Which energy providers tended to have the largest number of forced outages?
    
<p>

    
<img src = "https://media.istockphoto.com/id/1281821795/photo/market-stock-graph-and-information-with-city-light-and-electricity-and-energy-facility-banner.jpg?s=612x612&w=0&k=20&c=RSN5LqeMW28HW10aA190_DWR5YJ5tG2wixHFPBV3uZE=">

In [28]:
%%sql
SELECT COUNT(*) AS Total_Number_Outages, SUM(CASE WHEN Outage_Reason='Forced' THEN 1 ELSE 0 END) AS Total_Number_Forced_Outage_Events, ROUND(((SUM(CASE WHEN Outage_Reason='Forced' THEN 1 ELSE 0 END)*1.0)/COUNT(*)*100), 2) As Pct_Outage_Forced
FROM AEMR_Outage_Table
WHERE Status='Approved'
Group BY Year

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Number_Forced_Outage_Events,Pct_Outage_Forced
1931,1264,65.46
2171,1622,74.71


It's clear to see now that `Forced Outages` are problematic. Not only are they the only outage type that generates financial losses as the Outage is unplanned, it seems there is a number of Energy Participants who have been having a significantly high number of Outages.


Let's break our analysis down into Macro and Micro Analysis.
The total gives us the Overall Duration a participant is offline / has lost energy, however, it doesn't tell us how *frequently* this occurs. In other words, if we have one or two very big outages, it might contribute to very large totals.

However an <b> average </b> can help us identify how big these Outages might really be, spread across the year!


In [38]:
%%sql
SELECT COUNT(*) AS Total_Number_Outages, ROUND(SUM((ABS(JULIANDAY(End_Time)-JULIANDAY(Start_Time)))),2) AS Total_Duration_In_Days, ROUND(SUM(ENERGY_LOST_MW)) AS Total_Energy_Lost, Outage_Reason, Participant_Code, Facility_Code, Year
FROM AEMR_Outage_Table
WHERE Status='Approved'
Group BY Outage_reason, Participant_Code, Facility_Code, Year
ORDER BY Total_Energy_Lost DESC
LIMIT 20

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Duration_In_Days,Total_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
490,33.65,21640.0,Forced,AURICON,AURICON_PNJ_U1,2017
227,240.69,19327.0,Forced,GW,BW1_GREENWATERS_G2,2017
317,120.6,15751.0,Forced,GW,BW1_GREENWATERS_G2,2016
157,129.6,13771.0,Forced,MELK,MELK_G7,2016
208,15.06,10696.0,Forced,AURICON,AURICON_PNJ_U1,2016
177,404.15,10285.0,Forced,MELK,MELK_G7,2017
85,392.25,9669.0,Scheduled (Planned),MELK,MELK_G7,2016
69,34.06,9093.0,Forced,PMC,PMC_AG,2016
70,482.58,7499.0,Scheduled (Planned),MELK,MELK_G7,2017
46,87.02,6965.0,Scheduled (Planned),AURICON,AURICON_PNJ_U1,2016


In [49]:
%%sql
SELECT ROUND(AVG((ABS(JULIANDAY(End_Time)-JULIANDAY(Start_Time)))),2) AS Avg_Duration_In_Days, ROUND(AVG(ENERGY_LOST_MW),2) AS Avg_Energy_Lost, Outage_Reason, Participant_Code, Facility_Code, Year
FROM AEMR_Outage_Table
WHERE Status='Approved' AND Outage_Reason='Forced'
Group BY Outage_reason, Participant_Code, Facility_Code, Year
ORDER BY Avg_Energy_Lost DESC, Year DESC
LIMIT 5


 * sqlite:///AEMR.db
Done.


Avg_Duration_In_Days,Avg_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
1.11,149.0,Forced,COLLGAR,COLLGAR_WF1,2016
0.04,141.21,Forced,PMC,PMC_AG,2017
0.49,131.78,Forced,PMC,PMC_AG,2016
0.83,87.71,Forced,MELK,MELK_G7,2016
1.06,85.14,Forced,GW,BW1_GREENWATERS_G2,2017


In [57]:
%%sql
SELECT  ROUND(AVG(ENERGY_LOST_MW),2) AS Avg_Energy_Lost, ROUND(SUM(ENERGY_LOST_MW)) AS Total_Energy_Lost, ROUND((SUM(Energy_Lost_MW))/(SELECT SUM(ENERGY_LOST_MW) FROM AEMR_Outage_Table WHERE Status='Approved' AND Outage_Reason='Forced')*100,2) AS Pct_Energy_Lost, Outage_Reason, Participant_Code, Facility_Code, Year
FROM AEMR_Outage_Table
WHERE Status='Approved' AND Outage_Reason='Forced'
Group BY Outage_reason, Participant_Code, Facility_Code, Year
ORDER BY Total_Energy_Lost DESC, Year DESC
LIMIT 100

 * sqlite:///AEMR.db
Done.


Avg_Energy_Lost,Total_Energy_Lost,Pct_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
44.16,21640.0,14.21,Forced,AURICON,AURICON_PNJ_U1,2017
85.14,19327.0,12.69,Forced,GW,BW1_GREENWATERS_G2,2017
49.69,15751.0,10.34,Forced,GW,BW1_GREENWATERS_G2,2016
87.71,13771.0,9.04,Forced,MELK,MELK_G7,2016
51.42,10696.0,7.02,Forced,AURICON,AURICON_PNJ_U1,2016
58.11,10285.0,6.75,Forced,MELK,MELK_G7,2017
131.78,9093.0,5.97,Forced,PMC,PMC_AG,2016
72.61,5882.0,3.86,Forced,PJRH,PJRH_GT11,2016
141.21,5648.0,3.71,Forced,PMC,PMC_AG,2017
29.17,5017.0,3.29,Forced,TRMOS,TIWEST_COG1,2017


In [6]:
%%sql
SELECt *
FROM (
SELECT Participant_Code, Facility_Code, Description_of_Outage, ROUND (SUM(ENERGY_LOST_MW)) as Total_Energy_Lost,ROUND((SUM(Energy_Lost_MW))/(SELECT SUM(ENERGY_LOST_MW) FROM AEMR_Outage_Table WHERE Status='Approved' AND Outage_Reason='Forced')*100,2) AS Pct_Energy_Lost, RANK() OVER (PARTITION BY Participant_Code, Facility_Code ORDER BY SUM(ENERGY_LOST_MW) DESC) AS Rank
FROM AEMR_Outage_Table
WHERE Status='Approved' AND Outage_Reason='Forced' AND Participant_Code IN ('MELK', 'AURICON', 'GW')
GROUP BY Participant_Code, Facility_Code, Description_of_Outage)
WHERE Rank=1

 * sqlite:///AEMR.db
Done.


Participant_Code,Facility_Code,Description_of_Outage,Total_Energy_Lost,Pct_Energy_Lost,Rank
AURICON,AURICON_PNJ_U1,Full unit trip,6034.0,3.96,1
GW,BW1_GREENWATERS_G2,Operational Issues caused real time forced outage.,28688.0,18.84,1
MELK,MELK_G7,Safety Issues,1100.0,0.72,1
